# Import Modules

In [19]:
import pandas as pd
import numpy as np
import networkx as nx
from tqdm import tqdm
import pickle as pkl

In [20]:
all_nodes = set()
for i in tqdm(range(0, 31)):
    day = i % 31 + 1
    month = i // 31 + 7
    file_name = '2020-' + str(month).zfill(2) + '-' + str(day).zfill(2) + '.gexf'
    file_path = f'contact_graphs/{file_name}'

    G = nx.read_gexf(file_path)
    all_nodes = all_nodes.union(set(G.nodes()))

node_mapping = {node:i for i, node in enumerate(all_nodes)}
# pickle the node mapping
with open('node_mapping.pkl', 'wb') as f:
    pkl.dump(node_mapping, f)

100%|██████████| 31/31 [00:02<00:00, 11.51it/s]


In [21]:
# create a csv file with the following columns: source,destination,timestamp,state_label,w

# df = pd.DataFrame(columns=['source', 'destination', 'timestamp', 'state_label', 'w'])

dfs = []
# for i in tqdm(range(0, 62)):
for i in tqdm(range(0, 31)):
    day = i % 31 + 1
    month = i // 31 + 7
    file_name = '2020-' + str(month).zfill(2) + '-' + str(day).zfill(2) + '.gexf'
    file_path = f'contact_graphs/{file_name}'

    G = nx.read_gexf(file_path)
    # iterate through all edges
    # for edge in G.edges():
    #     # add edge to dataframe
    #     source = edge[0]
    #     destination = edge[1]
    #     timestamp = i
    #     state_label = 0
    #     w = 0
    dfs.append(pd.DataFrame([{'source': node_mapping[edge[0]], 'destination': node_mapping[edge[1]], 'timestamp': i, 'state_label': 0, 'w': 0} for edge in G.edges()]))

df = pd.concat(dfs)
# sources = set(df['source']).union(set(df['destination']))
# convert = {node:i for i, node in enumerate(sources)}
# df = df.replace(convert)

csv_file_path = 'data.csv'
df.to_csv(csv_file_path, index=False)



100%|██████████| 31/31 [00:02<00:00, 10.93it/s]
